In [ ]:
#r "nuget: FSharp.Stats, 0.4.3"
#r "nuget: BioFSharp, 2.0.0-beta5"
#r "nuget: BioFSharp.IO, 2.0.0-beta5"
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Deedle, 2.5.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"
#r "nuget: ARCtrl"
#r "nuget: ARCtrl.NET, 1.0.5"
#r "nuget: ARCtrl.QueryModel, 1.0.5"

open System.IO
open Deedle
open BioFSharp
open FSharpAux
open FSharp.Stats
open Plotly.NET
open ARCtrl
open ARCtrl.NET
open ARCtrl.QueryModel
open ARCtrl.ISA

# NB08c Absolute Quantification

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/CSBiology/BIO-BTE-06-L-7/gh-pages?filepath=NB08c_Absolute_Quantification_BN.ipynb)

[Download Notebook](https://github.com/CSBiology/BIO-BTE-06-L-7/releases/download/NB08c/NB08c_Absolute_Quantification_BN.ipynb)

Finally, after careful peptide ion selection, quality control and assuring that our label efficiency allows accurate for quantifications, we can start to
calculate ratios between rbcL and RbcS. Since we start again by getting access to our data and its description, this notebook will start off familiar!


## I. Reading the data
As promised, we start this notebook with the output of the previous analysis, this notebook assumes that the data from *NB06b Data Access and Quality Control* is stored in a .txt


In [ ]:
type PeptideIon = 
    {|
        ProteinGroup    : string  
        Synonyms        : string
        StringSequence  : string
        PepSequenceID   : int
        Charge          : int
    |}

//This is the filepath you chose in *NB08a Data Access and Quality Control*
// let filePath = @"C:\yourPath\testOut.txt"
let filePath = @"Your\Path\Here"

let qConcatDataFiltered =
    Frame.ReadCsv(path = filePath, separators = "\t")
    // StringSequence is the peptide sequence
    |> Frame.indexRowsUsing (fun os -> 
        {|
            ProteinGroup    = os.GetAs<string>("ProteinGroup"); 
            Synonyms        = os.GetAs<string>("Synonym")
            StringSequence  = os.GetAs<string>("StringSequence");
            PepSequenceID   = os.GetAs<int>("PepSequenceID");
            Charge          = os.GetAs<int>("Charge");
        |}
    )
    |> Frame.filterRows (fun k s -> String.contains "QProt_newPS" k.ProteinGroup)

let inline printIndexedData (f: Frame<{| Charge: int; PepSequenceID: int; ProteinGroup: string; StringSequence: string; Synonyms: string |},string>) =
    f
    |> Frame.mapRowKeys (fun k -> $"{k.ProteinGroup},{k.Synonyms},{k.StringSequence},{k.PepSequenceID},{k.Charge}")
    |> fun f -> f.Print()

## II. Extracting ratios and calculating comparisons.

Next, we want to look at the stoichiometry between rbcL and rbcS. First, we extract the ratios from our curated results file and plot them.


In [ ]:
let sliceQuantColumns quantColID frame = 
    frame
    |> Frame.filterCols (fun ck os -> ck |> String.contains ("." + quantColID))
    |> Frame.mapColKeys (fun ck -> ck.Split('.') |> Array.item 0)


let intensities = sliceQuantColumns "Quant_Light" qConcatDataFiltered

ratios
|> Frame.getRow {|Charge = 2; PepSequenceID = 1538853; ProteinGroup = "Cre02.g120100.t1.2;Cre02.g120150.t1.2;QProt_newPS"; StringSequence = "AFPDAYVR"; Synonyms = "RBCS1;RBCS2"|}

let intensityRbcL =
    intensities
    |> Frame.filterRows (fun r s -> r.Synonyms |> String.contains "rbcL")

let intensityRbcS =
    intensities
    |> Frame.filterRows (fun r s -> r.Synonyms |> String.contains "RBCS")

let medianRbcL =
    intensityRbcL
    |> Frame.getNumericCols
    |> Series.mapValues Stats.median
    |> Series.observations

let medianRbcS =
    intensityRbcS
    |> Frame.getNumericCols
    |> Series.mapValues Stats.median
    |> Series.observations


In [ ]:
[
    Chart.Column (medianRbcL)
    |> Chart.withTraceInfo "rbcL"
    Chart.Column (medianRbcS)
    |> Chart.withTraceInfo "rbcS"
]
|> Chart.combine


Since we are interested in the ratio between rbcL and rbcS, we just divide the rbcL intensity through the intensity of rbcS and compare the result with values from the literature.

In [ ]:
(medianRbcL, medianRbcS)
||> Seq.map2 (fun (s1,v1) (s2,v2) -> s1, v1 / v2)
|> Chart.Column


Do your results differ from the literature? Why? Discuss.
